In [12]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import os
import matplotlib.pyplot as plt

In [13]:
model_mn2 = MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)

model_mn2.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                              

In [14]:
layer_name = 'block_15_add'
feature_extractor = Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [15]:
diseases_names = []

for diseases in os.listdir('datasets/encoded/PlantVillage/train'):
    diseases_names += [diseases.split('.npy')[0]]

diseases_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(diseases_names)}

In [16]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f'datasets/encoded/PlantVillage/{set_kind}')):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f'datasets/encoded/PlantVillage/{set_kind}/{file}')
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[38] val/Tomato___Tomato_Yellow_Leaf_Curl_Virus.npyite.npypy

In [17]:
n_classes = len(diseases_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]


X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [18]:
input_layer = tf.keras.layers.Input(shape=[7, 7, 160], name="submodel_inputs")

block1 = tf.keras.layers.Conv2D(filters=320, kernel_size=1, use_bias=False, name="block_1_1_conv2d")(input_layer)
block1 = tf.keras.layers.BatchNormalization(name="block_1_1_bn")(block1)
block1 = tf.keras.layers.ReLU(name="block_1_1_relu")(block1)
block1 = tf.keras.layers.DepthwiseConv2D(kernel_size=3, padding='same', use_bias=False, name="block_1_2_depthwise")(block1)
block1 = tf.keras.layers.BatchNormalization(name="block_1_2_bn")(block1)
block1 = tf.keras.layers.ReLU(name="block_1_2_relu")(block1)
block1 = tf.keras.layers.Conv2D(filters=160, kernel_size=1, use_bias=False, name="block_1_3_conv2d")(block1)
block1 = tf.keras.layers.BatchNormalization(name="block_1_3_bn")(block1)
block1 = tf.keras.layers.ReLU(name="block_1_3_relu")(block1)

se_block1 = tf.keras.layers.GlobalAveragePooling2D(name="se_block_1_gap2d")(input_layer)
se_block1 = tf.keras.layers.Dense(units=24, name="se_block_1_1_dense")(se_block1)
se_block1 = tf.keras.layers.ReLU(name="se_block_1_1_relu")(se_block1)
se_block1 = tf.keras.layers.Dense(units=160, name="se_block_1_2_dense")(se_block1)
se_block1 = tf.keras.layers.Activation("sigmoid", name="se_block_1_2_sigmoid")(se_block1)

multiplication1 = tf.keras.layers.Multiply()([block1, se_block1])
addition1 = tf.keras.layers.Add()([input_layer, multiplication1])

block4 = tf.keras.layers.Conv2D(filters=720, kernel_size=1, use_bias=False, name="block_3_1_cond2d")(addition1)
block4 = tf.keras.layers.BatchNormalization(name="block_3_1_bn")(block4)
block4 = tf.keras.layers.ReLU(name="block_3_2_relu")(block4)

final_part = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling2d")(block4)
final_part = tf.keras.layers.Dropout(rate=0.8, name="dropout")(final_part)
output_layer = tf.keras.layers.Dense(units=39, activation="softmax", kernel_initializer="he_normal", name="predictions")(final_part)


model = Model(input_layer, output_layer)

In [19]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)

In [20]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.66, patience=4)

idx = np.arange(X_train.shape[0])
np.random.shuffle(idx)

X_train_shuffled = X_train[idx]
y_train_shuffled = y_train[idx]

# Free some precious RAM
del X_train
del instances['train']

In [21]:
filepath = 'model_squeeze_excite_resnet.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                monitor='val_loss',
                                                verbose=1,
                                                save_best_only=True,
                                                mode='min')

model.fit(
    X_train_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, checkpoint]
)

Epoch 1/10
1455/1455 [==============================] - ETA: 0s - loss: 0.7606 - accuracy: 0.7799         